<a href="https://colab.research.google.com/github/satheeshbhukya/Assign-Fantasy-game/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
folders = [
    "/content/character_model",  # Final saved model
    "/content/logs",             # Training logs
    "/content/api",              # (Optional) FastAPI or Flask app files
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)


In [ ]:
!apt-get install tree -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (72.2 kB/s)
Selecting previously unselected package tree.
(Reading database ... 126213 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!tree -d /content/

/content/
├── api
├── character_model
├── logs
└── sample_data

4 directories


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:

from datasets import load_dataset

# Load the full dataset
df = load_dataset("NousResearch/CharacterCodex")

# Check a sample
print(df["train"][0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.35k [00:00<?, ?B/s]

character_codex.json:   0%|          | 0.00/11.2M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'media_type': 'Webcomics', 'genre': 'Fantasy Webcomics', 'character_name': 'Alana', 'media_source': 'Saga', 'description': 'Alana is one of the main characters from the webcomic "Saga." She is a strong-willed and fiercely protective mother who is on the run with her family in a war-torn galaxy. The story blends elements of fantasy and science fiction, creating a rich and complex narrative.', 'scenario': "You are a fellow traveler in the galaxy needing help, and Alana offers her assistance while sharing stories of her family's struggles and triumphs."}


In [ ]:
print(len(df["train"]))


15939


In [ ]:
for i in range(3):
    print("Name:", df["train"][i]["character_name"])
    print("Description:", df["train"][i]["description"])
    print("Scenario:", df["train"][i]["scenario"])
    print("-" * 50)

Name: Alana
Description: Alana is one of the main characters from the webcomic "Saga." She is a strong-willed and fiercely protective mother who is on the run with her family in a war-torn galaxy. The story blends elements of fantasy and science fiction, creating a rich and complex narrative.
Scenario: You are a fellow traveler in the galaxy needing help, and Alana offers her assistance while sharing stories of her family's struggles and triumphs.
--------------------------------------------------
Name: Yilin
Description: Yilin is a young nun from the Hengshan Sect in Jin Yong's novel "The Smiling, Proud Wanderer." Known for her innocence and kindness, she becomes friends with the protagonist Linghu Chong. Her gentle nature often puts her at odds with the violent world of martial arts.
Scenario: You are a fellow disciple of the Hengshan Sect seeking Yilin's comfort and advice after a particularly brutal conflict. Her gentle demeanor and compassionate words provide solace in a harsh wor

In [ ]:
def format_row(example):
    prompt = (
        f"Name: {example['character_name']}\n"
        f"Media Type: {example['media_type']}\n"
        f"Genre: {example['genre']}\n"
        f"Source: {example['media_source']}\n"
        f"Description: {example['description']}\n\n"
        f"Scenario:"
    )
    return {"text": f"{prompt} {example['scenario']}"}

formatted = df["train"].map(format_row)


Map:   0%|          | 0/15939 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

tokenized = formatted.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=256), batched=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

Map:   0%|          | 0/15939 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

model = GPT2LMHeadModel.from_pretrained("distilgpt2")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./character_model",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

<ipython-input-11-e9f360de8d45>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: satheeshb0508 (satheesh-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,2.451500
1000,2.293400
1500,2.217900
2000,2.176500
2500,2.134700
3000,2.107500
3500,2.089800
4000,2.055000
4500,1.944600
5000,1.939800


Step,Training Loss
500,2.451500
1000,2.293400
1500,2.217900
2000,2.176500
2500,2.134700
3000,2.107500
3500,2.089800
4000,2.055000
4500,1.944600
5000,1.939800


TrainOutput(global_step=11955, training_loss=1.9742835121043105, metrics={'train_runtime': 2676.8395, 'train_samples_per_second': 17.863, 'train_steps_per_second': 4.466, 'total_flos': 3123606680764416.0, 'train_loss': 1.9742835121043105, 'epoch': 3.0})

In [ ]:
# Save the fine-tuned model and tokenizer
trainer.save_model("/content/character_model")
tokenizer.save_pretrained("/content/character_model")

# Save a training completion log
with open("/content/logs/train_log.txt", "w") as f:
    f.write("Training completed successfully.\nModel: distilgpt2\nEpochs: 3")


In [ ]:
from transformers import pipeline

prompt = "Name: Alana\nContext: A brave warrior protecting her family in a war-torn galaxy.\nScenario:"

# Before training
raw_model = pipeline("text-generation", model="distilgpt2", tokenizer=tokenizer)
print("🔹 Before fine-tuning:\n", raw_model(prompt, max_length=50, do_sample=True)[0]['generated_text'])

# After training
trained_model = pipeline("text-generation", model="/content/character_model", tokenizer=tokenizer)
print("🔹 After fine-tuning:\n", trained_model(prompt, max_length=50, do_sample=True)[0]['generated_text'])


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Device set to use cuda:0


🔹 Before fine-tuning:
 Name: Alana
Context: A brave warrior protecting her family in a war-torn galaxy.
Scenario: The conflict between Alana and Akash, she’s a force.
The Emperor Tatar’s rule is
🔹 After fine-tuning:
 Name: Alana
Context: A brave warrior protecting her family in a war-torn galaxy.
Scenario: The user meets Aali who is in need of a medical treatment. They discuss their desire to protect their homeworld and negotiate a


In [ ]:
!pip install transformers datasets torch fastapi uvicorn nest-asyncio pyngrok -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
!pip install gradio -q



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 76.2 MB/s eta 0:00:00


In [ ]:
%%writefile /content/api/main.py

import gradio as gr
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from datetime import datetime
import logging

# Setup logging
logging.basicConfig(filename="/content/logs/gradio_requests.log", level=logging.INFO)

# Load fine-tuned model
model = GPT2LMHeadModel.from_pretrained("/content/character_model")
tokenizer = GPT2Tokenizer.from_pretrained("/content/character_model")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define a safe wrapper with logging and error handling
def generate_scenario(prompt):
    try:
        result = generator(prompt, max_length=200, do_sample=True)
        response = result[0]["generated_text"]
        # Log prompt and response
        logging.info(f"[{datetime.now()}] Prompt: {prompt} | Output: {response[:80]}...")
        return response
    except Exception as e:
        logging.error(f"[{datetime.now()}] ERROR: {str(e)}")
        return f"Error generating response: {str(e)}"

# Gradio interface
demo = gr.Interface(
    fn=generate_scenario,
    inputs="text",
    outputs="text",
    title="Fantasy Character Scenario Generator",
    description="Enter a fantasy character's background to generate an RPG-style story snippet."
)



Overwriting /content/api/main.py


In [ ]:
from api.main import demo  # Importing demo from main.py

# Launch Gradio
demo.launch(share=True)

Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://79e11e398084565efc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
